In [1]:
import pandas as pd
import geopandas as gpd
import keplergl

In [2]:
# read csv
df = pd.read_csv('data/Worldwide-Earthquake-database.csv')
df.head()

,I_D,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
0,1,No,-2150,NaN,NaN,NaN,NaN,,NaN,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Yes,-2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,No,-2000,NaN,NaN,NaN,NaN,NaN,18.0,7.1,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
3,5877,Yes,-1610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
4,8,No,-1566,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# lat and lon to numeric, errors converted to nan
df['LONGITUDE'] = pd.to_numeric(df.LONGITUDE, errors='coerce')
df['LATITUDE'] = pd.to_numeric(df.LATITUDE, errors='coerce')# drop rows with missing lat, lon, and intensity
df.dropna(subset=['LONGITUDE', 'LATITUDE', 'INTENSITY'], inplace=True)# convert tsunami flag from string to int
df['FLAG_TSUNAMI'] = [1 if i=='Yes' else 0 for i in df.FLAG_TSUNAMI.values]

In [4]:
kepler_map = keplergl.KeplerGl(height=400)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [5]:
kepler_map.add_data(data=df, name="earthquakes")

In [6]:
kepler_map

KeplerGl(data={'earthquakes': {'index': [1, 2, 4, 5, 9, 11, 14, 16, 18, 20, 23, 24, 25, 27, 35, 36, 37, 39, 41…

In [12]:
config = kepler_map.config

In [13]:
kepler_map.save_to_html(file_name='earthquake.html', 
                        data={"earthquakes": df}, config=config)

Map saved to earthquake.html!


In [16]:
block = gpd.read_file('data/block-outlines.geojson')
block.dropna(inplace=True)
graffiti = gpd.read_file('data/graffiti.geojson')
graffiti.dropna(inplace=True)

In [17]:
# join datasets
graf_block = gpd.sjoin(block, graffiti, how='inner', op='contains')# create new indexes
graf_block.reset_index(inplace=True)
graf_block.head()

,index,geometry,index_right,count,geo_local_area
0,9,"POLYGON ((-123.10282 49.26973, -123.10280 49.2...",1998,5,Mount Pleasant
1,9,"POLYGON ((-123.10282 49.26973, -123.10280 49.2...",4242,1,Mount Pleasant
2,10,"POLYGON ((-123.10956 49.26796, -123.10956 49.2...",7035,2,Mount Pleasant
3,10,"POLYGON ((-123.10956 49.26796, -123.10956 49.2...",7604,2,Mount Pleasant
4,10,"POLYGON ((-123.10956 49.26796, -123.10956 49.2...",7508,1,Mount Pleasant


In [20]:
graf_block = graf_block.dissolve(by='index', aggfunc='sum')
data_dict = {"graffiti": graf_block, "block": block}

graffiti_map = keplergl.KeplerGl(height=500, 
                                 data=data_dict, 
                                 config=config)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [22]:
graffiti_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'n7vvrz', 'type': '…